In [1]:
%pip install nltk


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


# 1. Naives Bayes Theorem

Flow:

1. Read data:
```py
positive = open("positive.txt", "r").read()
negative = open("negative.txt", "r").read()
```

2. Tokenize datanya
Karena data yang diread akan dipindahkan ke dalam variable dalam tipe String. Kita harus tokenize dulu untuk bagian pre-processing datanya.
```py
list_words = word_tokenize(positive) + word_tokenize(negative)
```

3. Bersihin kata-kata

- Stopwords
```py
list_words = [word for word in list_words if word.lower() not in eng_stopwords]
```
- Punctuation
```py
list_words = [word for word in list_words if word.lower() not in string.punctuation]
```
- Ngilangin angka
```py
list_words = [word for word in list_words if word.isalpha()] # append ke list_words kalau word adalah sebuah alphabet
```

4. Perkecil range kata-kata agar time complexitynya lebih cepat
```py
fd = FreqDist(list_words)
```
Nah, disini kita cukup ambil most common's wordnya dia aja
```py
list_words = [word.word for word in fd.most_common(100)] # ngambil 100 kata most-common aja.
# word.word biar ngambil katanya doang.
```

5. Labeling sentences (pisahin kategori positif & negatif reviews)
```py
labeled_sentence = []
for sentence in positive.split("\n"):
    labeled_sentence.append((sentence, "pos"))

for sentence in negative.split("\n"):
    labeled_sentence.append((sentence, "neg"))
```

6. Membuat dataset (dari kata2 + labeled_sentence yang udah kita buat)
```py
dataset = []
for sentence, label in labeled_sentence:
    dict = {}
    # word tokenize sentencesnya
    words = word_tokenize(sentence)
    # masukin ke dalam dictionarynya
    for feature in list_words:
        key = feature # kata
        value = feature in words # kita mau cek aja sih ada ga sih kata-kata dalam kalimat kita yang ada dalam list of words bersih yang udah kita buat.
        dict[key] = value # contoh {"love" : False, "damn" : True}
    # kalau dictionarynya udah dibuat, kita masukin dia ke dataset yg udah kita buat
    dataset.append((dict, label)) # masukin dictionary + labelnya (positive/negative)
```
Kenapa kayak gini? Karena kita mau bikin dataset yang isinya tuh kurang lebih kata-kata (ada/engganya) di kalimat yang positif/negatif. Karena balik lagi kita mau pake perhitungan bayes theorem yang mana kita akan hitung probabilitas setiap katanya.

7. Shuffle kata-katanya karena isinya sekarang tuh pasti positif terus ke negatif.
```py
import random
random.shuffle(dataset)
```
Kenapa kita shuffle? Karena pada dasarnya ketika kita training kita bakal bagi datasetnya berdasarkan berapa persen berapa persen kan, nah agar hasilnya lebih akurat, kita shuffle dulu.

8. Bagi datasetnya
```py 
counter = int(len(dataset) * 0.7) # pembagian dataset 70% training, 30% testing
training_data = dataset[:counter]
testing_data = dataset[counter:]
```

9. Lakukan algoritma naives bayes

- Import dulu
```py
from nltk.classify import NaiveBayesClassifier, accuracy
```
- Buat objectnya + masukin data trainingnya
```py
classify = NaiveBayesClassifier.train(training_data)
```
- Cek akurasi
```py
accuracy(classifier, testing_data) # dibanding dengan testing data kita
```
- Kali 100 buat dapet persentasenya

Notes: tambahin most common word aja buat ningkatin akurasi. (semakin kotor dataset, semakin kecil persentase ke akuratannya)

10. Buat modelnya

- import picklenya
```py
import pickle
```
- open filenya
```py
file = open("model.pickle", "wb")
```
- masukin data classifiernya ke dalam sebuah file
```py
pickle.dump(classifier, file)
```

11. Terus cara pake modelnya gimana?

- open file picklenya pake mode "rb"
```py
file = open("model.pickle", "rb")
```
- load ke dalam classifier
```py
classifier = pickle.load(file)
```
- Terakhir jangan lupa di close
```py
file.close()
```
- Kalau mau by-input bisa langsung
```py
review = input()
words = word_tokenize(review)
classifier = classifier.classify(FreqDist(words)) # dia minta parameternya words yang udah di freq dist.
```